In [564]:
import pandas as pd
import requests
import re
import time
pd.set_option("display.max_columns", None)

##### Hacemos llamada inicial para saber si funciona y tenemos acceso a la página.
##### Establecemos parametros de la busqueda
##### Cambiamos parametros varias veces para buscar cada género por año ya que la API solo nos permite estos parametros de búsqueda sin darnos resultados incorrectos.

In [565]:


url_endpoint = "https://moviesdatabase.p.rapidapi.com/titles"

querystring =  {"genre":"Comedy","titleType":"movie", "year" :"2024"}    # obtuvimos estos parametros en la pagina API

headers = {
	"X-RapidAPI-Key": "1db777625dmshdce6ed30bc0ec06p198b62jsn978c35c52050",
	"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

llamada = requests.get(url_endpoint, headers=headers, params=querystring)

print(llamada.json())

llamada = requests.get(url_endpoint, headers=headers, params=querystring)

print(llamada.json())
resultados_llamada = llamada.json()

{'page': 1, 'next': '/titles?titleType=movie&year=2023&genre=Comedy&page=2', 'entries': 10, 'results': [{'_id': '61e58c128a5a6a599f43cfef', 'id': 'tt10151854', 'primaryImage': {'id': 'rm332475393', 'width': 675, 'height': 1000, 'url': 'https://m.media-amazon.com/images/M/MV5BNjBmMzQ4MDItZGNiMi00OWJmLTg3MDktMTViZmI0ODRhNzJkXkEyXkFqcGdeQXVyNjMxMTE2Mjg@._V1_.jpg', 'caption': {'plainText': 'Helen Mirren, Djimon Hounsou, Lucy Liu, Adam Brody, Meagan Good, Rachel Zegler, Zachary Levi, D.J. Cotrona, Grace Caroline Currey, and Ross Butler in Shazam! Fury of the Gods (2023)', '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': 'Shazam! Fury of t

##### Hemos visto que la llamada.json contiene 5 elementos: 'page' (el numero de la pagina), 'next' (la direcion exacta de la pagina), ##### 'entries' (el numero de peliculas en la pagina) y 'results' que contiene toda la información que nesitamos.

##### Ahora tenemos que crear un bucle que pase a la siguiente página despues de obtener la información.

In [566]:
def llamada_paginada(url_endpoint, headers, params):
    pagina = 1
    data_peliculas = []
    while pagina <= 10: # Aqui limitamos el numero de paginas que vamos a llamar.
        params['page'] = pagina
        llamada = requests.get(url_endpoint, headers=headers, params=params)
        
        if llamada.status_code == 200:
            llamada= llamada.json()
            if 'results' in llamada:
                data_peliculas.extend(llamada['results'])  # extraemos los datos de la clave 'resutls'
                print(f"Página {pagina}: {len(llamada['results'])} películas extraídas.")
            else:
                print("No hay más resultados. Finalizando la búsqueda.")
                break
            # Comprueba si existe otra pagina.
            if 'next' in llamada and llamada['next']:
                pagina += 1  # sigue a la siguiente pagina.
                time.sleep(1)  # Espera entre llamadas a la API para evitar sobrecarga y que nos restringan acceso a la pagina.
            else:
                print("Ultima pagina extraida.")
                break
        else:
            print(f"Error al buscar datos: {llamada.status_code} {llamada.reason}")
            break

    return data_peliculas
        
resultados_paginados = llamada_paginada(url_endpoint, headers=headers, params=querystring)        
        

Página 1: 10 películas extraídas.
Página 2: 10 películas extraídas.
Página 3: 10 películas extraídas.
Página 4: 10 películas extraídas.
Página 5: 10 películas extraídas.
Página 6: 10 películas extraídas.
Página 7: 10 películas extraídas.
Página 8: 10 películas extraídas.
Página 9: 10 películas extraídas.
Página 10: 10 películas extraídas.


##### Visualizamos resultados

In [567]:

print(type(resultados_llamada))
resultados_llamada

<class 'dict'>


{'page': 1,
 'next': '/titles?titleType=movie&year=2023&genre=Comedy&page=2',
 'entries': 10,
 'results': [{'_id': '61e58c128a5a6a599f43cfef',
   'id': 'tt10151854',
   'primaryImage': {'id': 'rm332475393',
    'width': 675,
    'height': 1000,
    'url': 'https://m.media-amazon.com/images/M/MV5BNjBmMzQ4MDItZGNiMi00OWJmLTg3MDktMTViZmI0ODRhNzJkXkEyXkFqcGdeQXVyNjMxMTE2Mjg@._V1_.jpg',
    'caption': {'plainText': 'Helen Mirren, Djimon Hounsou, Lucy Liu, Adam Brody, Meagan Good, Rachel Zegler, Zachary Levi, D.J. Cotrona, Grace Caroline Currey, and Ross Butler in Shazam! Fury of the Gods (2023)',
     '__typename': 'Markdown'},
    '__typename': 'Image'},
   'titleType': {'displayableProperty': {'value': {'plainText': '',
      '__typename': 'Markdown'},
     '__typename': 'DisplayableTitleTypeProperty'},
    'text': 'Movie',
    'id': 'movie',
    'isSeries': False,
    'isEpisode': False,
    'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}],
    'canHaveEpisodes': Fal

In [568]:
def organizar_pelis(resultados_json):
    peli_data = []
    for i in resultados_json:
        # Extraer los datos necesarios
        peli_id = i['id']
        peli_type = i['titleType']['text'] if 'titleType' in i else 'Desconocido'
        peli_nombre = i['titleText']['text'] if 'titleText' in i else 'Desconocido'
        
        # Manejo seguro de releaseDate para evitar AttributeError
        release_date = i.get('releaseDate', None)  # Usa None como valor por defecto
        if release_date is None:  # Si releaseDate es None, usar valores por defecto
            peli_año = 'Desconocido'
            peli_mes = 'Desconocido'
        else:
            peli_año = release_date.get('year', 'Desconocido')
            peli_mes = release_date.get('month', 'Desconocido')
        
        # Crear el diccionario con los datos 
        movie_dict = {
            'Tipo': peli_type,
            'Nombre': peli_nombre,
            'Año de Estreno': peli_año,
            'Mes de Estreno': peli_mes,
            'ID de la Película': peli_id
        }
        peli_data.append(movie_dict)
    
    print(peli_data)
    # Convertir la lista de diccionarios en un DataFrame
    return pd.DataFrame(peli_data)



##### 'resultados_llamada' contiene un diccionario con una clave para los resultado llamada 'results', necesitamos acceder a esa parte de la respuesta.
##### Debemos llamar a la función 'organizar_pelis' con resultados_llamada['results']

In [569]:
df_pelis = organizar_pelis(resultados_llamada['results'])

[{'Tipo': 'Movie', 'Nombre': 'Shazam! Fury of the Gods', 'Año de Estreno': 2023, 'Mes de Estreno': 3, 'ID de la Película': 'tt10151854'}, {'Tipo': 'Movie', 'Nombre': 'Rapture Camp', 'Año de Estreno': 2023, 'Mes de Estreno': 5, 'ID de la Película': 'tt10178004'}, {'Tipo': 'Movie', 'Nombre': "Creepers 'R Us", 'Año de Estreno': 2023, 'Mes de Estreno': 9, 'ID de la Película': 'tt10305228'}, {'Tipo': 'Movie', 'Nombre': 'Bad Boy', 'Año de Estreno': 2023, 'Mes de Estreno': 4, 'ID de la Película': 'tt10351180'}, {'Tipo': 'Movie', 'Nombre': 'Miraculous: Filmen om Ladybug & Cat Noir', 'Año de Estreno': 2023, 'Mes de Estreno': 7, 'ID de la Película': 'tt10364034'}, {'Tipo': 'Movie', 'Nombre': 'Os Perebas', 'Año de Estreno': 2023, 'Mes de Estreno': 11, 'ID de la Película': 'tt10750578'}, {'Tipo': 'Movie', 'Nombre': 'Next Goal Wins', 'Año de Estreno': 2024, 'Mes de Estreno': 1, 'ID de la Película': 'tt10767052'}, {'Tipo': 'Movie', 'Nombre': 'Al Reves', 'Año de Estreno': 2023, 'Mes de Estreno': 2, '

In [570]:
df_pelis

,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película
0,Movie,Shazam! Fury of the Gods,2023,3,tt10151854
1,Movie,Rapture Camp,2023,5,tt10178004
2,Movie,Creepers 'R Us,2023,9,tt10305228
3,Movie,Bad Boy,2023,4,tt10351180
4,Movie,Miraculous: Filmen om Ladybug & Cat Noir,2023,7,tt10364034
5,Movie,Os Perebas,2023,11,tt10750578
6,Movie,Next Goal Wins,2024,1,tt10767052
7,Movie,Al Reves,2023,2,tt10810852
8,Movie,Ant-Man and the Wasp: Quantumania,2023,2,tt10954600
9,Movie,Reception,2023,6,tt11170966


##### Guardamos busquedas 'Drama' por año en archivos csv. para luego combinarlos

In [571]:
# df_pelis.to_csv("drama_2010_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2011_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2012_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2013_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2015_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2016_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2017_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2018_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2019_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2020_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2021_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2022_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2023_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("drama_2024_10_paginas.csv", index=False, encoding='utf-8')


##### Guardamos busquedas 'Comedia' por año en archivos csv. para luego combinarlos

In [572]:
# df_pelis.to_csv("comedia_2010_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2011_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2012_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2013_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2014_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2015_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2016_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2017_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2018_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2019_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2020_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2021_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2022_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2023_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("comedia_2024_10_paginas.csv", index=False, encoding='utf-8')

##### Guardamos busquedas 'Acción' por año en archivos csv. para luego combinarlos

In [573]:
# df_pelis.to_csv("accion_2010_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2011_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2012_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2013_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2014_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2015_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2016_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2017_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2018_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2019_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2020_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2021_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2022_10_paginas.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2023_10_paginas.csv", index=False, encoding='utf-8')
df_pelis.to_csv("accion_2024_10_paginas.csv", index=False, encoding='utf-8')